# 🚀 02: LangGraph 고급 통합 패턴

## 📚 학습 목표
01-langgraph-fundamentals에서 배운 모든 패턴을 조합하여 실무에서 바로 사용할 수 있는 완전한 시스템을 구축합니다.

### 🔗 사전 학습 필요
이 노트북을 시작하기 전에 `01-langgraph-fundamentals.ipynb`를 먼저 완료하세요:
- ✅ Routing 패턴: LLM이 판단하는 지능적 라우팅
- ✅ Fan-out/Fan-in 패턴: 병렬 처리로 효율성 극대화  
- ✅ Summarization 패턴: 긴 대화를 지능적으로 요약
- ✅ Human in the Loop 패턴: AI 처리에 인간 검토 추가

## 🏆 실무 통합 예제 - 모든 패턴의 완벽한 조합

### 💼 실무 시나리오: 지능형 고객 서비스 시스템
1. **고객 질문 분류** (Routing): "기술 문의", "환불 요청", "일반 상담" 등으로 자동 분류
2. **병렬 전문 처리** (Fan-out/Fan-in): 각 분야별 전문 팀이 병렬로 조사 후 통합
3. **대화 기록 관리** (Summarization): 긴 상담 내역을 자동으로 요약해서 효율적 관리  
4. **최종 승인 과정** (Human in the Loop): 중요한 응답은 시니어 직원이 검토 후 승인

### 🔄 통합 워크플로우
**사용자 질문** → **지능형 분류** → **전문팀 병렬 조사** → **결과 통합** → **인간 검토** → **최종 응답**

### 🎯 비즈니스 가치
- **효율성**: 자동 분류와 병렬 처리로 처리 시간 단축
- **품질**: 각 분야 전문성 + 인간 검토로 높은 품질 보장
- **확장성**: 새로운 카테고리나 전문 영역 쉽게 추가
- **안전성**: 중요한 결정에는 반드시 인간 승인
- **지속가능성**: 대화 요약으로 메모리 효율성 유지

In [ ]:
# 🚀 필요한 라이브러리 설치 및 EXAONE 모델 로드
!pip install -q langgraph langchain langchain-teddynote
!pip install -q grandalf matplotlib networkx pyppeteer
# 시각화를 위한 추가 패키지들
!pip install -q graphviz pygraphviz
!pip install -q pydot
!pip install -q git+https://github.com/lgai-exaone/transformers@add-exaone4
!pip install -q torch accelerate

from typing import TypedDict, List, Dict, Any, Annotated
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import interrupt, Command
from langchain_core.messages import HumanMessage, AIMessage
import json
import random
import time
import torch
import gc
import re
import uuid
import operator
from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
warnings.filterwarnings('ignore')

# 시각화 관련 라이브러리들
try:
    from langchain_teddynote.graphs import visualize_graph
    print("✅ langchain-teddynote 시각화 모듈 로드 성공!")
except ImportError:
    print("⚠️  langchain-teddynote 시각화 모듈 로드 실패")

try:
    from IPython.display import Image, display
    print("✅ IPython 디스플레이 모듈 로드 성공!")
except ImportError:
    print("⚠️  IPython 디스플레이 모듈 로드 실패")

try:
    import graphviz
    print("✅ graphviz 모듈 로드 성공!")
except ImportError:
    print("⚠️  graphviz 모듈 로드 실패 - pip install graphviz 필요")

print("✅ 라이브러리 import 완료!")

# 🤖 EXAONE 모델 로드
MODEL_NAME = "LGAI-EXAONE/EXAONE-4.0-1.2B"

print(f"🚀 EXAONE-4.0-1.2B 모델 로드 시작: {MODEL_NAME}")

# EXAONE-4.0-1.2B 모델 및 토크나이저 로드 (CPU 호환)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,  # CPU 호환을 위해 float32 사용
    trust_remote_code=True
)

print("✅ EXAONE-4.0-1.2B 모델 로드 성공!")

# GPU 메모리 최적화
def clear_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

def pure_exaone_inference(messages_or_prompt):
    """🔥 EXAONE 대화 맥락 지원 함수"""
    clear_gpu_memory()
    
    try:
        if isinstance(messages_or_prompt, str):
            messages = [{"role": "user", "content": messages_or_prompt}]
        else:
            messages = messages_or_prompt
        
        input_ids = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=100,
                temperature=0.1,
                do_sample=True,
                top_p=0.9,
                pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        
        generated_ids = outputs[0][input_ids.shape[-1]:]
        ai_response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
        
        clear_gpu_memory()
        return ai_response if ai_response else "안녕하세요!"
        
    except Exception as e:
        clear_gpu_memory()
        print(f"🚨 EXAONE 추론 오류: {e}")
        return "죄송해요, 다시 말씀해 주시겠어요?"

print("✅ EXAONE 모델 설정 완료!")
print("🚀 고급 통합 패턴 시스템 준비 완료!")

# 1. 통합 State 정의

모든 패턴의 데이터를 포괄하는 통합 State를 정의합니다.

In [ ]:
# 🏆 통합 고객 서비스 시스템 State 정의 (모든 패턴 통합)
class IntegratedCustomerServiceState(TypedDict):
    # 기본 정보
    customer_query: str                    # 고객 질문
    conversation_history: Annotated[List[str], operator.add]  # 대화 기록
    
    # Routing 패턴 정보
    query_category: str                    # 질문 카테고리 (technical/policy/service)
    routing_confidence: float              # 분류 신뢰도
    
    # Fan-out/Fan-in 패턴 정보  
    technical_analysis: str                # 기술팀 분석 결과
    policy_guidance: str                   # 정책팀 분석 결과
    service_recommendation: str            # 고객서비스팀 분석 결과
    integrated_response: str               # 통합된 응답
    
    # Summarization 패턴 정보
    conversation_summary: str              # 대화 요약
    needs_summary: bool                    # 요약 필요 여부
    
    # Human in the Loop 패턴 정보
    requires_human_review: bool            # 인간 검토 필요 여부
    human_feedback: str                    # 인간 검토자 피드백
    approval_status: str                   # 승인 상태
    
    # 최종 결과
    final_customer_response: str           # 최종 고객 응답

print("🏆 통합 고객 서비스 시스템 State 정의 완료!")
print("   📊 모든 LangGraph 패턴의 데이터를 포괄하는 통합 구조")
print("   🔄 Routing, Fan-out/Fan-in, Summarization, Human in the Loop 지원")

# 2. Routing 패턴 구현

고객 질문을 지능적으로 분류하는 라우팅 시스템을 구현합니다.

In [ ]:
# 🧠 Routing 패턴 - 지능형 고객 질문 분류 (수정됨)

def classify_customer_query(state: IntegratedCustomerServiceState) -> str:
    """고객 질문을 지능적으로 분류하여 경로 결정"""
    query = state["customer_query"]
    
    routing_prompt = f"""
다음 고객 질문을 분석해서 적절한 카테고리로 분류해주세요:

고객 질문: {query}

카테고리 옵션:
1. "technical" - 기술적 문제, 오류, 버그, 설정 관련
2. "policy" - 환불, 취소, 정책, 규정 관련
3. "service" - 일반 문의, 정보 요청, 고객 서비스 관련

가장 적절한 카테고리를 하나만 선택해서 답변해주세요.
"""
    
    llm_response = pure_exaone_inference(routing_prompt)
    
    # LLM 응답에서 카테고리 추출
    if "technical" in llm_response.lower():
        category = "technical"
    elif "policy" in llm_response.lower():
        category = "policy"
    else:
        category = "service"
    
    print(f"🧠 라우팅 완료: {query[:30]}... → {category}")
    return category

def routing_node(state: IntegratedCustomerServiceState) -> IntegratedCustomerServiceState:
    """라우팅 결과를 상태에 저장하는 노드"""
    category = classify_customer_query(state)
    
    return {
        "query_category": category,
        "routing_confidence": 0.9,
        "requires_human_review": category == "policy"  # 정책 관련은 인간 검토 필요
    }

print("🧠 Routing 패턴 구현 완료!")
print("   🎯 EXAONE 기반 지능형 질문 분류")
print("   📋 technical/policy/service 3개 카테고리")
print("   🔧 classify_customer_query: 경로 결정 함수")
print("   📊 routing_node: 상태 업데이트 노드")

# 3. Fan-out/Fan-in 패턴 구현

여러 전문팀이 병렬로 분석하고 결과를 통합하는 시스템을 구현합니다.

In [ ]:
# 🔄 Fan-out/Fan-in 패턴 - 전문팀 병렬 분석

def technical_team_node(state: IntegratedCustomerServiceState) -> IntegratedCustomerServiceState:
    """기술팀 전문 분석"""
    query = state["customer_query"]
    category = state.get("query_category", "")
    
    if category == "technical":
        prompt = f"""
기술팀 관점에서 다음 고객 문의를 분석해주세요:

고객 질문: {query}

기술적 관점에서:
- 문제 원인 분석
- 해결 방법 제시
- 예방 조치 안내

전문적이고 정확한 기술 지원을 제공해주세요.
"""
    else:
        prompt = f"기술팀 검토: {query} - 기술적 이슈 없음"
    
    technical_analysis = pure_exaone_inference(prompt)
    print(f"🔧 기술팀 분석 완료")
    
    return {"technical_analysis": technical_analysis}

def policy_team_node(state: IntegratedCustomerServiceState) -> IntegratedCustomerServiceState:
    """정책팀 전문 분석"""
    query = state["customer_query"]
    category = state.get("query_category", "")
    
    if category == "policy":
        prompt = f"""
정책팀 관점에서 다음 요청을 검토해주세요:

고객 요청: {query}

정책 검토 사항:
- 정책 적용 가능성
- 필요한 절차와 서류
- 예상 처리 시간
- 대안 제시

회사 정책에 따른 정확한 안내를 제공해주세요.
"""
    else:
        prompt = f"정책팀 검토: {query} - 특별한 정책 이슈 없음"
    
    policy_guidance = pure_exaone_inference(prompt)
    print(f"📋 정책팀 분석 완료")
    
    return {"policy_guidance": policy_guidance}

def service_team_node(state: IntegratedCustomerServiceState) -> IntegratedCustomerServiceState:
    """고객서비스팀 전문 분석"""
    query = state["customer_query"]
    
    prompt = f"""
고객서비스팀 관점에서 다음 고객 문의에 대한 응답을 준비해주세요:

고객 질문: {query}

고객 서비스 관점에서:
- 고객 감정과 니즈 파악
- 친절하고 공감하는 응답
- 추가 지원 방안
- 고객 만족도 향상 방법

고객 중심적이고 따뜻한 서비스를 제공해주세요.
"""
    
    service_recommendation = pure_exaone_inference(prompt)
    print(f"💝 고객서비스팀 분석 완료")
    
    return {"service_recommendation": service_recommendation}

def fan_in_aggregator_node(state: IntegratedCustomerServiceState) -> IntegratedCustomerServiceState:
    """3개 팀의 분석 결과를 통합"""
    query = state["customer_query"]
    technical = state.get("technical_analysis", "")
    policy = state.get("policy_guidance", "")
    service = state.get("service_recommendation", "")
    
    integration_prompt = f"""
다음 고객 질문에 대한 3개 전문팀의 분석 결과를 통합해서 완전한 응답을 만들어주세요:

고객 질문: {query}

🔧 기술팀 분석:
{technical}

📋 정책팀 분석:
{policy}

💝 고객서비스팀 분석:
{service}

위 3개 팀의 전문 의견을 종합하여:
1. 고객 문의에 대한 완전한 해답
2. 단계별 해결 방안
3. 추가 지원 정보
4. 친절하고 전문적인 톤

통합된 고품질 고객 응답을 작성해주세요.
"""
    
    integrated_response = pure_exaone_inference(integration_prompt)
    print(f"🔗 전문팀 분석 통합 완료")
    
    return {"integrated_response": integrated_response}

print("🔄 Fan-out/Fan-in 패턴 구현 완료!")
print("   🔧 technical_team_node: 기술적 문제 전문 분석")
print("   📋 policy_team_node: 정책 및 규정 전문 검토")
print("   💝 service_team_node: 고객 서비스 관점 분석")
print("   🔗 fan_in_aggregator_node: 모든 분석 결과 통합")

# 4. Summarization 패턴 구현

긴 대화 기록을 지능적으로 요약하는 시스템을 구현합니다.

In [ ]:
# 📚 Summarization 패턴 - 대화 기록 관리

def conversation_summarizer_node(state: IntegratedCustomerServiceState) -> IntegratedCustomerServiceState:
    """대화 기록을 요약"""
    history = state.get("conversation_history", [])
    query = state["customer_query"]
    integrated_response = state.get("integrated_response", "")
    
    # 현재 대화를 기록에 추가
    current_conversation = f"고객: {query}\nAI 응답: {integrated_response}"
    
    if len(history) >= 3:  # 3개 이상의 대화가 누적되면 요약
        # 기존 대화 + 현재 대화를 모두 요약
        all_conversation = "\n\n".join(history + [current_conversation])
        
        summary_prompt = f"""
다음 고객 서비스 대화 내용을 핵심만 간단히 요약해주세요:

대화 내용:
{all_conversation}

요약 포함 사항:
- 고객의 주요 요청사항
- 제공된 해결방안
- 현재 진행 상황
- 추가 필요한 조치

간결하고 명확한 요약을 작성해주세요.
"""
        
        summary = pure_exaone_inference(summary_prompt)
        print(f"📚 대화 요약 완료")
        
        return {
            "conversation_summary": summary,
            "conversation_history": [f"[요약] {summary}", current_conversation],  # 요약 + 최근 대화만 유지
            "needs_summary": False
        }
    else:
        print(f"📚 요약 불필요 (대화 {len(history) + 1}턴)")
        return {
            "conversation_summary": "대화 시작 단계",
            "conversation_history": [current_conversation],
            "needs_summary": False
        }

print("📚 Summarization 패턴 구현 완료!")
print("   📝 긴 대화 자동 요약으로 메모리 효율성 유지")
print("   🎯 핵심 정보 보존하면서 토큰 사용량 최적화")

# 5. Human in the Loop 패턴 구현

중요한 결정에 인간의 검토와 승인을 추가하는 시스템을 구현합니다.

In [ ]:
# 👥 Human in the Loop 패턴 - 최종 승인 과정 (동작 명확화)

def human_review_node(state: IntegratedCustomerServiceState) -> IntegratedCustomerServiceState:
    """시니어 직원의 검토 및 승인"""
    query = state["customer_query"]
    category = state.get("query_category", "")
    integrated_response = state.get("integrated_response", "")
    requires_review = state.get("requires_human_review", False)
    
    print(f"👥 Human in the Loop 단계:")
    print(f"   📂 카테고리: {category}")
    print(f"   🔍 검토 필요: {requires_review}")
    
    if requires_review:
        print(f"   🛑 정책 관련 문의로 시니어 검토가 필요합니다!")
        print(f"   📧 AI 통합 응답:")
        print(f"   {integrated_response[:200]}...")
        
        # 🛑 실제 Human in the Loop - 워크플로우 중단!
        feedback = interrupt({
            "message": "🚨 시니어 직원 검토가 필요합니다",
            "reason": f"{category} 카테고리는 인간 승인이 필수입니다",
            "customer_query": query,
            "ai_response": integrated_response,
            "instructions": "응답을 검토하고 승인/수정 피드백을 주세요. '승인' 또는 구체적인 수정 요청을 입력하세요."
        })
        
        return {
            "human_feedback": feedback,
            "approval_status": "reviewed"
        }
    else:
        print(f"   ⚡ {category} 카테고리는 자동 승인됩니다")
        return {
            "human_feedback": "자동 승인",
            "approval_status": "auto_approved"
        }

def final_response_node(state: IntegratedCustomerServiceState) -> IntegratedCustomerServiceState:
    """최종 응답 확정"""
    integrated_response = state.get("integrated_response", "")
    human_feedback = state.get("human_feedback", "")
    approval_status = state.get("approval_status", "auto_approved")
    
    print(f"✅ 최종 응답 생성:")
    print(f"   📋 승인 상태: {approval_status}")
    
    if approval_status == "reviewed" and human_feedback:
        if "승인" in human_feedback:
            final_response = integrated_response
            print("   ✅ 시니어 승인 완료 - 원본 응답 사용")
        else:
            # 피드백 반영해서 수정
            revision_prompt = f"""
다음 AI 응답을 시니어 직원의 피드백에 따라 수정해주세요:

원본 응답: {integrated_response}
시니어 피드백: {human_feedback}

피드백을 반영한 개선된 최종 응답을 작성해주세요.
"""
            final_response = pure_exaone_inference(revision_prompt)
            print("   🔄 피드백 반영 수정 완료")
    else:
        final_response = integrated_response
        print("   ⚡ 자동 승인 완료 - 원본 응답 사용")
    
    return {"final_customer_response": final_response}

print("👥 Human in the Loop 패턴 구현 완료!")
print("")
print("🎯 Human in the Loop 동작 규칙:")
print("   📋 정책(policy) 문의 → 🛑 interrupt() 발생 → 인간 검토 필수")
print("   🔧 기술(technical) 문의 → ⚡ 자동 승인 → 빠른 처리")
print("   💝 서비스(service) 문의 → ⚡ 자동 승인 → 빠른 처리")
print("")
print("💡 실제 운영 시나리오:")
print("   1. 정책 문의 시 워크플로우가 중단되고 __interrupt__ 키 반환")
print("   2. 시니어가 웹 인터페이스에서 검토 후 피드백 입력")
print("   3. Command(resume=피드백)으로 워크플로우 재시작")
print("   4. 피드백에 따라 응답 수정 또는 원본 응답 사용")

# 6. 통합 워크플로우 구성

이제 모든 패턴을 하나의 워크플로우로 연결하여 완전한 고객 서비스 시스템을 만들어보겠습니다.

In [ ]:
# 통합 워크플로우 생성 (수정됨: 진정한 Fan-out/Fan-in 패턴)

# StateGraph 생성
workflow = StateGraph(IntegratedCustomerServiceState)

# 모든 노드 추가
workflow.add_node("routing", routing_node)  # 라우팅 노드로 변경
workflow.add_node("technical_team", technical_team_node)
workflow.add_node("policy_team", policy_team_node)
workflow.add_node("service_team", service_team_node)
workflow.add_node("fan_in_aggregator", fan_in_aggregator_node)
workflow.add_node("conversation_summarizer", conversation_summarizer_node)
workflow.add_node("human_review", human_review_node)
workflow.add_node("final_response", final_response_node)

# 🔄 진정한 Fan-out/Fan-in 워크플로우 구성
workflow.add_edge(START, "routing")

# 🌟 Fan-out: 라우팅 후 모든 팀이 병렬로 분석 수행
workflow.add_edge("routing", "technical_team")
workflow.add_edge("routing", "policy_team") 
workflow.add_edge("routing", "service_team")

# 🔄 Fan-in: 모든 팀의 분석 결과를 통합
workflow.add_edge("technical_team", "fan_in_aggregator")
workflow.add_edge("policy_team", "fan_in_aggregator")
workflow.add_edge("service_team", "fan_in_aggregator")

# 순차 처리: 통합 → 요약 → 휴먼 리뷰 → 최종 응답
workflow.add_edge("fan_in_aggregator", "conversation_summarizer")
workflow.add_edge("conversation_summarizer", "human_review")
workflow.add_edge("human_review", "final_response")
workflow.add_edge("final_response", END)

# 체크포인터와 함께 컴파일
checkpointer = MemorySaver()
integrated_app = workflow.compile(checkpointer=checkpointer)

print("✅ 진정한 Fan-out/Fan-in 통합 워크플로우 완성!")
print("🔄 수정된 워크플로우 구조:")
print("   START → 라우팅 → [모든팀 병렬분석] → 통합 → 요약 → 휴먼리뷰 → 최종응답 → END")
print("")
print("🌟 주요 개선사항:")
print("   ✅ 라우팅 후 모든 팀이 병렬로 분석 (진정한 Fan-out)")
print("   ✅ 각 팀이 카테고리에 관계없이 자신의 관점에서 분석")
print("   ✅ Fan-in에서 모든 분석 결과를 종합하여 고품질 응답 생성")
print("   ✅ 타입 안전성 확보 및 명확한 역할 분리")

In [ ]:
# 📊 워크플로우 시각화 (수정됨 - 여러 방법 시도)
print("📊 통합 워크플로우 시각화")
print("=" * 50)
from langchain_teddynote.graphs import visualize_graph
visualize_graph(integrated_app)


# 테스트 시나리오 1: 기술 문의 (자동 승인 데모) - 개선됨
print("=== 테스트 시나리오 1: 기술 문의 ===")
print("")
print("🎯 이 시나리오의 목적:")
print("   ✅ 라우팅 패턴: '크래시' 키워드로 'technical' 카테고리 분류")
print("   ✅ Fan-out/Fan-in: 모든 팀이 병렬 분석 후 통합")
print("   ✅ 요약 패턴: 대화 기록 관리")
print("   ✅ Human in Loop: 기술 문의는 자동 승인 (interrupt 없음)")
print("")
print("📋 예상 실행 흐름:")
print("   1. 라우팅 → 'technical' 분류")
print("   2. 모든 팀 병렬 분석 (기술팀 상세, 다른 팀 간략)")
print("   3. 결과 통합 → 기술적 해결책 중심의 응답 생성")
print("   4. 대화 요약 → 첫 번째 대화로 요약 불필요")
print("   5. Human Review → 자동 승인 (requires_human_review=False)")
print("   6. 최종 응답 → 고객에게 완성된 기술 지원 제공")
print("")

initial_state = IntegratedCustomerServiceState(
    customer_query="앱이 자꾸 크래시가 나는데 어떻게 해결하나요?",
    conversation_history=[],
    query_category="",
    routing_confidence=0.0,
    technical_analysis="",
    policy_guidance="", 
    service_recommendation="",
    integrated_response="",
    conversation_summary="",
    needs_summary=False,
    requires_human_review=False,
    human_feedback="",
    approval_status="",
    final_customer_response=""
)

# 워크플로우 실행
config = {"configurable": {"thread_id": "test_technical_1"}}

print("🚀 워크플로우 실행 중...")
try:
    result = integrated_app.invoke(initial_state, config)
    
    print("")
    print("📋 실행 결과:")
    print(f"   📂 분류 카테고리: {result['query_category']}")
    print(f"   🔧 기술팀 분석: {result['technical_analysis'][:80]}...")
    print(f"   📋 정책팀 검토: {result['policy_guidance'][:60]}...")
    print(f"   💝 서비스팀 분석: {result['service_recommendation'][:60]}...")
    print(f"   📝 대화 요약: {result['conversation_summary']}")
    print(f"   👥 인간 검토: {result['human_feedback']}")
    print(f"   📋 승인 상태: {result.get('approval_status', 'Unknown')}")
    print(f"   💬 최종 응답: {result['final_customer_response'][:100]}...")
    
    # 안전한 인터럽트 체크
    if "__interrupt__" in result:
        print("")
        print("❌ 예상과 다름: interrupt가 발생했습니다!")
        print("🔍 기술 문의인데 인간 검토가 트리거되었습니다")
        
        # 인터럽트 데이터 안전하게 출력
        try:
            interrupt_info = result["__interrupt__"]
            print(f"🎯 인터럽트 데이터 타입: {type(interrupt_info)}")
            print(f"🎯 인터럽트 내용: {str(interrupt_info)[:100]}...")
        except Exception as int_err:
            print(f"⚠️ 인터럽트 데이터 처리 오류: {int_err}")
    else:
        print("")
        print("✅ 예상대로 작동: 기술 문의는 자동 승인으로 완료!")
        print("🎯 Human in the Loop가 올바르게 우회되었습니다")
        
    # 추가 검증
    print("")
    print("🔍 상세 검증:")
    print(f"   📋 requires_human_review: {result.get('requires_human_review', 'Not Set')}")
    if result.get('requires_human_review') == False:
        print("   ✅ 기술 문의는 올바르게 자동 승인 경로로 처리됨")
    else:
        print("   ⚠️ 예상과 다른 human_review 설정")
        
except Exception as e:
    print(f"❌ 워크플로우 실행 오류: {e}")
    print("🔍 디버깅 정보:")
    print(f"   📋 오류 타입: {type(e).__name__}")
    print(f"   📋 오류 메시지: {str(e)}")
    
print("\n" + "="*60)

In [ ]:
# 테스트 시나리오 2: 정책 문의 (Human in the Loop 활성화 데모) - 수정됨
print("=== 테스트 시나리오 2: 정책 문의 ===")
print("")
print("🎯 이 시나리오의 목적:")
print("   ✅ 라우팅 패턴: '환불' 키워드로 'policy' 카테고리 분류")
print("   ✅ Fan-out/Fan-in: 모든 팀이 병렬 분석 (정책팀 상세)")
print("   ✅ 요약 패턴: 기존 대화 기록과 함께 요약")
print("   ✅ Human in Loop: 정책 문의는 interrupt() 발생 → 인간 검토 필수!")
print("")
print("📋 예상 실행 흐름:")
print("   1. 라우팅 → 'policy' 분류")
print("   2. 모든 팀 병렬 분석 (정책팀 상세, 다른 팀 지원)")
print("   3. 결과 통합 → 정책 가이드라인 중심의 응답 생성")
print("   4. 대화 요약 → 기존 대화와 통합하여 요약")
print("   5. Human Review → interrupt() 발생! (__interrupt__ 키 반환)")
print("   6. 워크플로우 중단 → 시니어 검토 대기")
print("")

policy_state = IntegratedCustomerServiceState(
    customer_query="환불 정책이 어떻게 되나요? 구매한지 15일 됐는데 환불 가능한가요?",
    conversation_history=["이전에 비슷한 문의가 있었습니다."],
    query_category="",
    routing_confidence=0.0,
    technical_analysis="",
    policy_guidance="", 
    service_recommendation="",
    integrated_response="",
    conversation_summary="",
    needs_summary=False,
    requires_human_review=False,
    human_feedback="",
    approval_status="",
    final_customer_response=""
)

config2 = {"configurable": {"thread_id": "test_policy_1"}}

print("🚀 워크플로우 실행 중...")
try:
    result2 = integrated_app.invoke(policy_state, config2)
    
    print("")
    print("📋 실행 결과:")
    print(f"   📂 분류 카테고리: {result2['query_category']}")
    print(f"   📋 정책팀 가이던스: {result2['policy_guidance'][:80]}...")
    print(f"   📝 대화 요약: {result2['conversation_summary']}")
    
    if "__interrupt__" in result2:
        print("")
        print("✅ 예상대로 작동: interrupt가 발생했습니다!")
        print("🛑 정책 관련 문의로 시니어 검토가 필요합니다")
        
        # 🔧 안전한 인터럽트 데이터 처리
        print("📋 인터럽트 상세 정보:")
        try:
            interrupt_data = result2["__interrupt__"]
            
            # 데이터 타입별 처리
            if isinstance(interrupt_data, dict):
                print("   📊 인터럽트 데이터 타입: 딕셔너리")
                for key, value in interrupt_data.items():
                    if key == "ai_response":
                        print(f"   {key}: {str(value)[:100]}...")
                    else:
                        print(f"   {key}: {value}")
                        
            elif isinstance(interrupt_data, list):
                print("   📊 인터럽트 데이터 타입: 리스트")
                for i, item in enumerate(interrupt_data):
                    print(f"   인터럽트 {i+1}: {str(item)[:100]}...")
                    
            else:
                print("   📊 인터럽트 데이터 타입: 기타")
                print(f"   전체 데이터: {str(interrupt_data)[:200]}...")
                
        except Exception as e:
            print(f"   ⚠️ 인터럽트 데이터 파싱 오류: {e}")
            print(f"   🔍 원본 데이터 타입: {type(result2.get('__interrupt__', 'Not Found'))}")
            print(f"   🔍 원본 데이터: {str(result2.get('__interrupt__', 'Not Found'))[:100]}...")
        
        print("")
        print("💡 다음 단계 (실제 운영 시):")
        print("   1. 시니어가 웹 인터페이스에서 AI 응답 검토")
        print("   2. 승인 또는 수정 요청 입력")
        print("   3. Command(resume='피드백')으로 워크플로우 재시작")
        
        # 시뮬레이션: 시니어 승인 후 재시작
        print("")
        print("🎭 시니어 승인 시뮬레이션...")
        try:
            resume_result2 = integrated_app.invoke(
                Command(resume="정책에 따라 15일 이내이므로 환불 승인합니다."), 
                config2
            )
            print("✅ 워크플로우 재시작 완료!")
            print(f"💬 최종 고객 응답: {resume_result2['final_customer_response'][:150]}...")
        except Exception as resume_error:
            print(f"⚠️ 워크플로우 재시작 오류: {resume_error}")
        
    else:
        print("")
        print("❌ 예상과 다름: interrupt가 발생하지 않았습니다!")
        print("🔍 정책 문의인데 자동 승인되었을 가능성이 있습니다")
        print(f"   📋 requires_human_review: {result2.get('requires_human_review', 'Unknown')}")
        print(f"   📋 approval_status: {result2.get('approval_status', 'Unknown')}")
        print(f"💬 최종 응답: {result2['final_customer_response'][:150]}...")
        
except Exception as e:
    print(f"❌ 워크플로우 실행 오류: {e}")
    print("🔍 디버깅 정보:")
    print(f"   📋 오류 타입: {type(e).__name__}")
    print(f"   📋 오류 메시지: {str(e)}")

print("\n" + "="*60)
print("")
print("🎓 학습 포인트:")
print("   📌 기술 문의 = 자동 승인 = 빠른 처리")
print("   📌 정책 문의 = 인간 검토 = 신중한 처리")
print("   📌 interrupt()는 실제 워크플로우를 중단시키는 강력한 도구")
print("   📌 Command(resume)으로 중단된 워크플로우를 재시작할 수 있음")
print("   📌 인터럽트 데이터 구조는 환경에 따라 다를 수 있음")

# 8. 결론 및 학습 포인트

## 🎯 이번 실습에서 배운 것들

### 패턴 통합의 힘
- **라우팅 패턴**: 적절한 전문가에게 문의를 자동 분배
- **Fan-out/Fan-in 패턴**: 여러 관점에서 동시에 문제를 분석하고 통합
- **요약 패턴**: 긴 대화를 효율적으로 관리
- **Human in the Loop**: 중요한 결정에서 인간의 판단력 활용

### 실제 비즈니스 적용
이 통합 시스템은 다음과 같은 실제 시나리오에서 활용할 수 있습니다:
- 🎧 **고객 서비스 센터**: 문의를 자동으로 분류하고 전문가 검토 후 응답
- 🏥 **의료 상담**: 증상을 분석하고 의사의 최종 검토 후 권고사항 제공
- 💼 **법률 자문**: 법적 문제를 분석하고 변호사의 검토 후 조언 제공
- 🏭 **제품 지원**: 기술 문제를 자동 분류하고 전문가 승인 후 해결책 제공

### 다음 단계
- 더 복잡한 조건부 라우팅 구현
- 실시간 피드백 루프 추가
- 성능 모니터링 및 최적화
- 다양한 데이터 소스와의 통합
- 멀티모달 입력 (텍스트, 이미지, 음성) 지원

## 💡 핵심 인사이트
각각의 LangGraph 패턴은 강력하지만, 이들을 조합했을 때 진정한 가치가 나타납니다. 실제 비즈니스 문제는 단일 패턴으로 해결되지 않으며, 이런 통합 접근법이 필요합니다.

## 🚀 실무 적용 가이드
1. **단계적 접근**: 한 번에 모든 패턴을 구현하지 말고 점진적으로 추가
2. **비즈니스 우선순위**: 가장 중요한 비즈니스 문제부터 해결
3. **사용자 피드백**: 실제 사용자의 피드백을 통한 지속적 개선
4. **성능 모니터링**: 각 패턴의 성능을 모니터링하고 최적화
5. **확장성 고려**: 미래의 요구사항 변화에 대비한 유연한 설계

🎉 **축하합니다! 이제 LangGraph의 모든 핵심 패턴을 마스터하고 실무급 통합 시스템을 구축할 수 있습니다!**